# Data Collection — FBref

This notebook scrapes player-season statistics from FBref
(Big 5 European Leagues) and prepares a clean dataset
for downstream machine learning.

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup, Comment
from io import StringIO
import time


In [3]:
with open("../data/raw/2022_23_big5_players.html", "r", encoding="utf-8") as f:
    html = f.read()

soup = BeautifulSoup(html, "html.parser")

# Unhide tables inside comments
comments = soup.find_all(string=lambda t: isinstance(t, Comment))
for c in comments:
    soup.append(BeautifulSoup(c, "html.parser"))

table = soup.find("table", {"id": "stats_standard"})
df_standard = pd.read_html(StringIO(str(table)))[0]

In [4]:
df_standard.shape


(3004, 38)

In [5]:
df_standard.columns

MultiIndex([( 'Unnamed: 0_level_0',       'Rk'),
            ( 'Unnamed: 1_level_0',   'Player'),
            ( 'Unnamed: 2_level_0',   'Nation'),
            ( 'Unnamed: 3_level_0',      'Pos'),
            ( 'Unnamed: 4_level_0',    'Squad'),
            ( 'Unnamed: 5_level_0',     'Comp'),
            ( 'Unnamed: 6_level_0',      'Age'),
            ( 'Unnamed: 7_level_0',     'Born'),
            (       'Playing Time',       'MP'),
            (       'Playing Time',   'Starts'),
            (       'Playing Time',      'Min'),
            (       'Playing Time',      '90s'),
            (        'Performance',      'Gls'),
            (        'Performance',      'Ast'),
            (        'Performance',      'G+A'),
            (        'Performance',     'G-PK'),
            (        'Performance',       'PK'),
            (        'Performance',    'PKatt'),
            (        'Performance',     'CrdY'),
            (        'Performance',     'CrdR'),
            (       

In [14]:
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    ),
    "Accept-Language": "en-US,en;q=0.9",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Referer": "https://fbref.com/",
    "Connection": "keep-alive"
}


In [17]:
def get_fbref_table(url, table_id):

    time.sleep(10)

    res = requests.get(url, headers=HEADERS)
    res.raise_for_status()

    soup = BeautifulSoup(res.text, "html.parser")

    # FBref hides many tables inside HTML comments
    comments = soup.find_all(string=lambda text: isinstance(text, Comment))
    for c in comments:
        soup.append(BeautifulSoup(c, "html.parser"))

    table = soup.find("table", {"id": table_id})
    if table is None:
        raise ValueError(f"Table {table_id} not found")

    return pd.read_html(StringIO(str(table)))[0]


In [18]:
url = "https://fbref.com/en/comps/Big5/2022-2023/stats/players/2022-2023-Big-5-European-Leagues-Stats"

df_standard = get_fbref_table(url, "stats_standard")
df_standard.head()


HTTPError: 403 Client Error: Forbidden for url: https://fbref.com/en/comps/Big5/2022-2023/stats/players/2022-2023-Big-5-European-Leagues-Stats

In [7]:
url = "https://fbref.com/en/comps/Big5/2022-2023/stats/players/2022-2023-Big-5-European-Leagues-Stats"

pd.read_html(url, attrs={"id": "stats_standard"})[0].head()

len(tables)



HTTPError: HTTP Error 403: Forbidden